In [ ]:

%%capture

!pip install flair

In [ ]:
# Prepare data
data_path = "/content/drive/MyDrive/DataENG/"
exp_id = "EngEXP1crf"
save_dir = "/content/drive/MyDrive/DataENG/"+exp_id
model_name = 'xlm-roberta-base'
model_name = 'google/muril-base-cased'

lines = open(data_path+"data1.dis",'r').readlines()
labels = open(data_path+"data1.labels",'r').readlines()

In [ ]:
print(lines[:5])
print(labels[:5])

['send uhm send eta via whatsapp\n', 'send te text to heidi and include moms eta\n', 'send a text to um 5555555\n', 'send please send nancy an sms\n', 'i want to email zane this photo and cc um and cc zach\n']
['1 1 0 0 0 0\n', '0 1 0 0 0 0 0 0 0\n', '0 0 0 0 1 0\n', '1 0 0 0 0 0\n', '0 0 0 0 0 0 0 1 1 1 0 0 0\n']


In [ ]:
import random
temp = list(zip(lines, labels))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
lines, labels = list(res1), list(res2)

train_lines = lines[:int(0.8*len(lines))]
train_labels = labels[:int(0.8*len(labels))]
val_lines = lines[int(0.8*len(lines)):int(0.9*len(lines))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_lines = lines[int(0.9*len(lines)):]
test_labels = labels[int(0.9*len(labels)):]

In [ ]:
print(len(train_lines))
print(len(val_lines))
len(test_lines)

3180
397


398

In [ ]:
import pandas as pd
sentence_col = []
word_col = []
tag_col = []
for i in range(len(train_lines)):
  sent = train_lines[i][:-1]
  label = train_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_train = pd.DataFrame()
df_train['Sentence #'] = sentence_col
df_train['Word'] = word_col
df_train['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(val_lines)):
  sent = val_lines[i][:-1]
  label = val_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_val = pd.DataFrame()
df_val['Sentence #'] = sentence_col
df_val['Word'] = word_col
df_val['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(test_lines)):
  sent = test_lines[i][:-1]
  label = test_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_test = pd.DataFrame()
df_test['Sentence #'] = sentence_col
df_test['Word'] = word_col
df_test['Tag'] = tag_col

In [ ]:
df_train[:20]

,Sentence #,Word,Tag
0,Sentence: 1,swaralipi,0
1,None,bengali,0
2,None,is,0
3,None,any,0
4,None,system,0
5,None,used,0
6,None,in,0
7,None,sheet,0
8,None,music,0
9,None,in,0


In [ ]:
import os
os.makedirs("data_dir",exist_ok = True)

In [ ]:
with open("data_dir/train.txt",'w') as write_file:
  for i in range(len(df_train)):
    if(df_train['Sentence #'][i]!=None and i!=0):
      write_file.write("\n")
    write_file.write(df_train['Word'][i]+" "+df_train['Tag'][i]+"\n")


with open("data_dir/val.txt",'w') as write_file:
  for i in range(len(df_val)):
    if(df_val['Sentence #'][i]!=None and i!=0):
      write_file.write("\n")
    write_file.write(df_val['Word'][i]+" "+df_val['Tag'][i]+"\n")


with open("data_dir/test.txt",'w') as write_file:
  for i in range(len(df_test)):
    if(df_test['Sentence #'][i]!=None and i!=0):
      write_file.write("\n")
    write_file.write(df_test['Word'][i]+" "+df_test['Tag'][i]+"\n")

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0 : 'text', 1 : 'dc'}
# directory where the data resides
data_folder = 'data_dir/'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'val.txt')

2023-12-24 22:05:26,668 Reading data from data_dir
2023-12-24 22:05:26,670 Train: data_dir/train.txt
2023-12-24 22:05:26,673 Dev: data_dir/val.txt
2023-12-24 22:05:26,676 Test: data_dir/test.txt


In [ ]:
corpus.train[1004]

Sentence[16]: "i need to log my um log my 1 cup of grapes for my 2pm snack" → ["i"/0, "need"/0, "to"/0, "log"/1, "my"/1, "um"/1, "log"/0, "my"/0, "1"/0, "cup"/0, "of"/0, "grapes"/0, "for"/0, "my"/0, "2pm"/0, "snack"/0]

In [ ]:
print(len(corpus.train))

3180


In [ ]:
# tag to predict
label_type = 'dc'
# make tag dictionary from the corpus
#tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
#print(tag_dictionary)

label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

2023-12-24 22:05:28,364 Computing label dictionary. Progress:


0it [00:00, ?it/s]
3180it [00:00, 25537.03it/s]

2023-12-24 22:05:28,588 Dictionary created for label 'dc' with 2 values: 0 (seen 26655 times), 1 (seen 4864 times)
Dictionary with 2 tags: 0, 1


In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

embeddings = TransformerWordEmbeddings(model=model_name,
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=False,
                                       use_context=True,
                                       )

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

In [ ]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=label_dict,
                                       tag_type=label_type,
                                       use_crf=True,
                                       use_rnn=False)
print(tagger)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(tagger))
#238150676-238150660 = CRF
#240250884 - 238150660 = RNN

2023-12-24 22:06:09,151 SequenceTagger predicts: Dictionary with 2 tags: 0, 1
SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(197286, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(


In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train(save_dir,
              learning_rate=0.1,
              mini_batch_size=16,
              max_epochs=150)

2023-12-24 22:06:35,287 ----------------------------------------------------------------------------------------------------
2023-12-24 22:06:35,292 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(197286, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout

100%|██████████| 7/7 [05:15<00:00, 45.01s/it]

2023-12-24 22:45:03,144 DEV : loss 0.39718008041381836 - f1-score (micro avg)  0.8457
2023-12-24 22:45:03,165  - 0 epochs without improvement
2023-12-24 22:45:03,171 saving best model


2023-12-24 22:45:14,273 ----------------------------------------------------------------------------------------------------
2023-12-24 22:45:15,419 epoch 2 - iter 19/199 - loss 0.39481334 - time (sec): 1.14 - samples/sec: 2732.58 - lr: 0.100000 - momentum: 0.000000
2023-12-24 22:45:16,414 epoch 2 - iter 38/199 - loss 0.40244901 - time (sec): 2.13 - samples/sec: 2920.50 - lr: 0.100000 - momentum: 0.000000
2023-12-24 22:45:17,223 epoch 2 - iter 57/199 - loss 0.39951574 - time (sec): 2.94 - samples/sec: 3152.82 - lr: 0.100000 - momentum: 0.000000
2023-12-24 22:45:17,822 epoch 2 - iter 76/199 - loss 0.40484475 - time (sec): 3.54 - samples/sec: 3462.48 - lr: 0.100000 - momentum: 0.000000
2023-12-24 22:45:18,466 epoch 2 - iter 95/199 - loss 0.40883334 - time (sec): 4.18 - samples/sec: 3616.59 - lr: 0.100000 - momentum: 0.000000
2023-12-24 22:45:19,135 epoch 2 - iter 114/199 - loss 0.41007500 - time (sec): 4.85 - samples/sec: 3747.36 - lr: 0.100000 - momentum: 0.000000
2023-12-24 22:45:19,66

100%|██████████| 7/7 [00:01<00:00,  3.86it/s]

...

2023-12-24 23:00:06,090 DEV : loss 0.33115994930267334 - f1-score (micro avg)  0.8593
2023-12-24 23:00:06,120  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [9.765625e-05]
2023-12-24 23:00:06,123 ----------------------------------------------------------------------------------------------------
2023-12-24 23:00:06,126 learning rate too small - quitting training!
2023-12-24 23:00:06,128 ----------------------------------------------------------------------------------------------------
2023-12-24 23:00:06,130 Saving model ...


2023-12-24 23:00:10,426 Done.
2023-12-24 23:00:10,428 ----------------------------------------------------------------------------------------------------
2023-12-24 23:00:10,552 Loading model from best epoch ...
2023-12-24 23:00:17,954 SequenceTagger predicts: Dictionary with 4 tags: 0, 1, <START>, <STOP>


100%|██████████| 7/7 [05:55<00:00, 50.79s/it]

2023-12-24 23:06:14,294 
Results:
- F-score (micro) 0.8691
- F-score (macro) 0.5683
- Accuracy 0.8691

By class:
              precision    recall  f1-score   support

           0     0.8687    0.9975    0.9287      3530
           1     0.8875    0.1177    0.2079       603

    accuracy                         0.8691      4133
   macro avg     0.8781    0.5576    0.5683      4133
weighted avg     0.8715    0.8691    0.8235      4133

2023-12-24 23:06:14,296 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8691023469634648}

In [ ]:
sent1 = "Add sandwich ooh sandwich for lunch."
sent2 = "Add 700 calories a umm calorie protein hhhhhhhhhhhhh shake for breakfast."
sent3 = "The medium mmh medium burger I ate had 600 calories."
#sent4 = "मेरे द्वारा खाई गई कैलोरी को रिकॉर्ड करें"
#sent5 = "अब तक का मेरा वोह क्या कहते हैं कैलोरी सेवन है २१०० रिकॉर्ड करो"
sentences = []
sentences.append(sent1)
sentences.append(sent2)
sentences.append(sent3)
#sentences.append(sent4)
#sentences.append(sent5)

In [ ]:
sentences

['Add sandwich ooh sandwich for lunch.',
 'Add 700 calories a umm calorie protein hhhhhhhhhhhhh shake for breakfast.',
 'The medium mmh medium burger I ate had 600 calories.']

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load(save_dir+'/best-model.pt')
# create example sentence
for x in sentences:
  sentence = Sentence(x)
  # predict the tags
  model.predict(sentence)
  print(sentence.to_tagged_string())

2023-12-24 23:09:54,735 SequenceTagger predicts: Dictionary with 4 tags: 0, 1, <START>, <STOP>
Sentence[7]: "Add sandwich ooh sandwich for lunch." → ["Add"/0, "sandwich"/0, "ooh"/0, "sandwich"/0, "for"/0, "lunch"/0, "."/0]
Sentence[12]: "Add 700 calories a umm calorie protein hhhhhhhhhhhhh shake for breakfast." → ["Add"/0, "700"/0, "calories"/0, "a"/0, "umm"/0, "calorie"/0, "protein"/0, "hhhhhhhhhhhhh"/0, "shake"/0, "for"/0, "breakfast"/0, "."/0]
Sentence[11]: "The medium mmh medium burger I ate had 600 calories." → ["The"/0, "medium"/0, "mmh"/0, "medium"/0, "burger"/0, "I"/0, "ate"/0, "had"/0, "600"/0, "calories"/0, "."/0]
